In [19]:
import os
import sys
import pandas as pd

In [20]:
print os.listdir('.')

['.ipynb_checkpoints', 'ap_data.csv', 'client_data.csv', 'make_df.py', 'make_df.pyc', 'master_ap_data.csv', 'master_client_data.csv', 'master_csv.py', 'README.md', 'Untitled.ipynb']


In [28]:
ap_df = pd.read_csv('data_ap.csv')
client_df = pd.read_csv('data_client.csv')

In [43]:
times = client_df['First time seen']
sorted_times = times.sort_values(inplace=False).unique()

In [58]:
client_df['First time seen'] = client_df['First time seen'].apply(pd.to_datetime)
client_df['Last time seen'] = client_df['Last time seen'].apply(pd.to_datetime)

In [83]:
times = client_df['First time seen']

print type(times[0])

#print times.sort_values(inplace=False)[0]
#sorted_times = times.sort_values(inplace=False)
#start = sorted_times[0]
#print times - start
#print times

<class 'pandas.tslib.Timestamp'>
